In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1700679319815_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-1>

In [5]:
%pyspark
data_frame=spark.read.csv('s3://ebernalcdatalake/raw/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv',inferSchema=True,header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
%pyspark
print(data_frame.dtypes)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('fecha reporte web', 'string'), ('ID de caso', 'int'), ('Fecha de notificación', 'string'), ('Código DIVIPOLA departamento', 'int'), ('Nombre departamento', 'string'), ('Código DIVIPOLA municipio', 'int'), ('Nombre municipio', 'string'), ('Edad', 'int'), ('Unidad de medida de edad', 'int'), ('Sexo', 'string'), ('Tipo de contagio', 'string'), ('Ubicación del caso', 'string'), ('Estado', 'string'), ('Código ISO del país', 'int'), ('Nombre del país', 'string'), ('Recuperado', 'string'), ('Fecha de inicio de síntomas', 'string'), ('Fecha de muerte', 'string'), ('Fecha de diagnóstico', 'string'), ('Fecha de recuperación', 'string'), ('Tipo de recuperación', 'string'), ('Pertenencia étnica', 'int'), ('Nombre del grupo étnico', 'string')]

In [26]:
%pyspark
data_frame.select('Nombre departamento').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|Nombre departamento|
+-------------------+
|             BOGOTA|
|              VALLE|
|          ANTIOQUIA|
|          ANTIOQUIA|
|          ANTIOQUIA|
|          ANTIOQUIA|
|          CARTAGENA|
|             BOGOTA|
|             BOGOTA|
|             BOGOTA|
|             BOGOTA|
|              HUILA|
|              HUILA|
|              VALLE|
|               META|
|             BOGOTA|
|             BOGOTA|
|             BOGOTA|
|             BOGOTA|
|          ANTIOQUIA|
+-------------------+
only showing top 20 rows

In [11]:
%pyspark
data_frame = data_frame.withColumnRenamed('ID de caso', 'id de caso')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%pyspark
data_frame.select('id de caso').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|id de caso|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
|        10|
|        11|
|        12|
|        13|
|        14|
|        15|
|        16|
|        17|
|        18|
|        19|
|        20|
+----------+
only showing top 20 rows

In [18]:
%pyspark
from pyspark.sql import functions as F
department_case_counts = data_frame.groupBy('Nombre departamento').agg(F.count('*').alias('total_cases'))

top_departments = department_case_counts.sort(F.col('total_cases').desc()).limit(10)
top_departments.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+
|Nombre departamento|total_cases|
+-------------------+-----------+
|             BOGOTA|        401|
|              VALLE|        148|
|          ANTIOQUIA|        106|
|       CUNDINAMARCA|         49|
|          CARTAGENA|         39|
|          RISARALDA|         34|
|       BARRANQUILLA|         31|
|              HUILA|         30|
|            QUINDIO|         23|
|    NORTE SANTANDER|         19|
+-------------------+-----------+

In [21]:
%pyspark
daily_cases = data_frame.groupBy('fecha reporte web').agg(F.count('*').alias('total_cases'))
top_days = daily_cases.sort(F.col('total_cases').desc()).limit(10)

top_days.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------+
|fecha reporte web|total_cases|
+-----------------+-----------+
|31/3/2020 0:00:00|        107|
|24/3/2020 0:00:00|        105|
|30/3/2020 0:00:00|         96|
| 1/4/2020 0:00:00|         95|
|29/3/2020 0:00:00|         94|
|23/3/2020 0:00:00|         73|
|28/3/2020 0:00:00|         69|
|25/3/2020 0:00:00|         61|
|27/3/2020 0:00:00|         49|
|20/3/2020 0:00:00|         48|
+-----------------+-----------+

In [22]:
%pyspark
cases_municipio = data_frame.groupBy('Nombre municipio').agg(F.count('*').alias('total_cases'))
orderd = cases_municipio.sort(F.col('total_cases').desc()).limit(10)

orderd.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|Nombre municipio|total_cases|
+----------------+-----------+
|          BOGOTA|        401|
|            CALI|        101|
|        MEDELLIN|         63|
|       CARTAGENA|         39|
|    BARRANQUILLA|         31|
|           NEIVA|         27|
|         PEREIRA|         25|
|         PALMIRA|         22|
|      VALLEDUPAR|         16|
|         ARMENIA|         15|
+----------------+-----------+

In [25]:
%pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


def classify_age(edad):
    if edad > 50:
        return 'Viejo'
    else:
        return 'Joven'


classify_age_udf = udf(classify_age, StringType())


df_with_age_category = data_frame.withColumn('Edad categoria', classify_age_udf(data_frame['Edad']))


df_with_age_category.select('Edad', 'Edad categoria').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------+
|Edad|Edad categoria|
+----+--------------+
|  19|         Joven|
|  34|         Joven|
|  50|         Joven|
|  55|         Viejo|
|  25|         Joven|
|  27|         Joven|
|  85|         Viejo|
|  22|         Joven|
|  28|         Joven|
|  36|         Joven|
|  42|         Joven|
|  74|         Viejo|
|  68|         Viejo|
|  48|         Joven|
|  30|         Joven|
|  61|         Viejo|
|  73|         Viejo|
|  54|         Viejo|
|  54|         Viejo|
|  26|         Joven|
+----+--------------+
only showing top 20 rows